In [3]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate,KFold,ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [58]:
import pandas as pd

In [4]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,test_size=196,random_state=0)

In [6]:
# Linear regression on train
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train,X_train)
results = model.fit()

X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid-valid_pred)**2)

np.float64(23.61661706966988)

In [7]:
def evalMSE(terms, response, train, test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]

    X_test = mm.transform(test)
    y_test = test[response]
    
    results = sm.OLS(y_train,X_train).fit()
    test_pred = results.predict(X_test)

    return np.mean((y_test - test_pred)**2)

In [10]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower',degree)],'mpg',Auto_train,Auto_valid)
MSE    # Can be repeated with other random seeds for further assessment

array([23.61661707, 18.76303135, 18.79694163])

In [13]:
# Cross-Validation
# sklearn_sm is a wrapper that conencts the models from sm to the
# evaluation techniques in sklearn

hp_model = sklearn_sm(sm.OLS,MS(['horsepower']))
X,Y = Auto.drop(columns=['mpg']) , Auto['mpg']

cv_results = cross_validate(hp_model, X, Y, cv = Auto.shape[0]) # model which is an sklearn
                                         # model with fit() predict() and score()
                                         # methods, features, response, integer cv 
                                         # here for K-folds

cv_err = np.mean(cv_results['test_score'])

In [15]:
cv_err

np.float64(24.231513517929212)

In [18]:
# Check several poly fits

cv_error =np.zeros(5)

H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate (range(1,6)):
    X = np.power.outer(H,np.arange(d+1))
    M_CV = cross_validate(M,X,Y,cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03320428])

In [21]:
# Check several poly fits for K 10

cv_error = np.zeros(5)
cv = KFold(n_splits=10,shuffle=True,random_state=0
          )
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)

for i, d in enumerate (range(1,6)):
    X = np.power.outer(H,np.arange(d+1))
    M_CV = cross_validate(M,X,Y,cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])

cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848404, 19.13722016])

In [24]:
# The Bootsrap
Portfolio = load_data('Portfolio')
def alpha_func(D,idx):
    cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
    return ((cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))

In [25]:
alpha_func(Portfolio,range(100))

np.float64(0.57583207459283)

In [26]:
rng = np.random.default_rng(0)
alpha_func(Portfolio, rng.choice(100,100,replace=True))

np.float64(0.6074452469619004)

In [30]:
def boot_SE(func, D, n=None, B=1000,seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0    
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,n,replace=True)
        value = func(D,idx)
        first_ += value
        second_ += value**2
        return np.sqrt(second_ /B - (first_ /B)**2)

In [31]:
alpha_SE = boot_SE(alpha_func, Portfolio, B=1000,seed=0)

In [32]:
alpha_SE

np.float64(0.019199498387420112)

In [61]:
# Suing above for a regression model
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_,X_).fit().params

In [62]:
hp_func = partial(boot_OLS, MS(['horsepower']),'mpg')


In [68]:
# Note error in book, Auto by deafult does not have integer index so
# loc method does not work as expected. Cann add with reset_index

rng = np.random.default_rng(0)
np.array([hp_func(Auto.reset_index(),rng.choice(392,392,replace=True)) for _ in range(10)])

array([[39.88064456, -0.1567849 ],
       [38.73298691, -0.14699495],
       [38.31734657, -0.14442683],
       [39.91446826, -0.15782234],
       [39.43349349, -0.15072702],
       [40.36629857, -0.15912217],
       [39.62334517, -0.15449117],
       [39.0580588 , -0.14952908],
       [38.66688437, -0.14521037],
       [39.64280792, -0.15555698]])

In [69]:
hp_se = boot_SE(hp_func,Auto.reset_index(),B=1000,seed=10)

In [70]:
hp_se

intercept     1.318953
horsepower    0.005516
dtype: float64

In [71]:
quad_model = MS([poly('horsepower',2,raw=True)])
quad_func = partial(boot_OLS,quad_model,'mpg')
boot_SE(quad_func,Auto.reset_index(),B=1000)

intercept                                  1.662718
poly(horsepower, degree=2, raw=True)[0]    0.012315
poly(horsepower, degree=2, raw=True)[1]    0.000030
dtype: float64